In [ ]:
import requests
import csv

# API endpoint for listing all users
url = "https://api.clerk.dev/v1/users"

# Your Clerk API bearer token for authentication
bearer_token = "sk_test_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

# Headers for the request, including the Authorization header with your bearer token
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json",
}

# Initialize a list to store all users
all_users = []
offset = 0

while True:

    # Optional query parameters, adjust as needed
    params = {
        "offset": offset,
        "limit": 500,
        "order_by": "created_at"
    }

    # Make the GET request to the Clerk API
    response = requests.get(url, headers=headers, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        users = response.json()

        # Add the users to the list
        all_users.extend(users)
        offset += len(users)

        # Check if the number of users received is less than 500, indicating the end of pagination
        if len(users) < 500:
            break

        # Update the 'start_after' parameter to fetch the next page of results
        params["start_after"] = users[-1]["id"]
    else:
        # Handle errors
        print(f"Failed to retrieve users: {response.status_code}")
        print(response.text)
        break

# Now, 'all_users' contains all the users retrieved from pagination
print(f"Retrieved {len(all_users)} users.")

# Specify the filename
filename = "clerk_users.csv"

# Get headers automatically from the keys of the first user dictionary
# This assumes all dictionaries have the same structure
headers = list(all_users[0].keys())

# Add the "email" column header to the list of headers
headers.append("email")

# Write to CSV
with open(filename, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()  # Write the headers
    for user in all_users:
        # Get the first email address from the 'email_address' list
        email_address = user['email_addresses'][0]['email_address'] if user['email_addresses'] else ''
        user['email'] = email_address  # Add the 'email' field to the user dictionary
        writer.writerow(user)

print(f"CSV file has been created: {filename}")
print(f"Column headers: {headers}")